In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
from pathlib import Path
import csv
from sklearn.preprocessing import OneHotEncoder
from keras.optimizers import RMSprop
from collections import Counter
from itertools import repeat, chain

In [2]:
hp_df = pd.read_csv(Path('./hp_df.csv'))
hp_df.drop(columns='Unnamed: 0',inplace=True)
hp_df.head()

,word
0,THE
1,BOY
2,WHO
3,LIVED
4,Mr.


In [20]:
words=hp_df['word'].to_list()
words[0:5]

['THE', 'BOY', 'WHO', 'LIVED', 'Mr.']

In [57]:
word_sort = sorted(words, key = words.count, reverse = True)
word_sort[0:10]

['the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the', 'the']

In [47]:
word_sort.index("summer...”")

83162

In [76]:
word_sort[3651]

'the'

In [103]:
weighted_words = []
weighted_count = []
count = 0
for i in word_sort:
    if count == 0:
        weighted_words.append(i)
        count+=1
    elif word_sort.index(i) == 83162:
        #for j in weighted_count:
            #if j == count:
                #count = j + 0.00000000001 * word_sort.index(i)
                #break
        weighted_count.append(count)
        count = 1
        weighted_words.append(i)
        weighted_count.append(count)
    elif i != weighted_words[-1]:
        #for j in weighted_count:
            #if j == count:
                #count = j + 0.00000000001 * word_sort.index(i)
                #break
        weighted_count.append(count)
        count = 1
        weighted_words.append(i)
    else:
        count+=1

display(weighted_words[:10])
display(weighted_count[:10])
display(weighted_words[-10:])
display(weighted_count[-10:])

['the', 'and', 'to', 'a', 'Harry', 'of', 'was', 'he', 'in', 'his']

[3652, 2137, 1827, 1578, 1252, 1233, 1150, 1020, 898, 892]

['mustached,',
 'manner',
 'speaking,”',
 'day.”',
 'summer,',
 'holiday,”',
 'uncertainly',
 'unpleasant.',
 'spreading',
 'summer...”']

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [113]:
hp_df = pd.DataFrame(columns=['word','id'])
hp_df['word'] = weighted_words
hp_df['id'] = weighted_count
#hp_df.loc[hp_df['word'] == 'Potter']
hp_df.head(50)

,word,id
0,the,3652
1,and,2137
2,to,1827
3,a,1578
4,Harry,1252
5,of,1233
6,was,1150
7,he,1020
8,in,898
9,his,892


In [116]:
hp_df[hp_df.duplicated(subset='word')]

,word,id
25,Potter,1
26,Stone,1
27,Potter,1
28,Stone,1
29,Potter,3
...,...,...
35663,"sweater,",1
35664,"nothing,",1
35665,family!”,1
35666,"will,”",1


In [125]:
hp_df = hp_df.groupby('word').sum()
hp_df = hp_df.sort_values(by=['id'],ascending=False)
hp_df.head(50)

,id
word,
the,3652
and,2137
to,1827
a,1578
Harry,1252
of,1233
was,1150
he,1020
in,898


In [126]:
hp_df.to_csv('./hp_df_unique.csv')

In [3]:
hp_unique_df = pd.read_csv(Path('./hp_df_unique.csv'))
hp_unique_df.head()

,word,id
0,the,3652
1,and,2137
2,to,1827
3,a,1578
4,Harry,1252


In [8]:
id_l = hp_unique_df['id'].to_list()
id_l[:5]

[3652, 2137, 1827, 1578, 1252]

In [9]:
# Makes all values unique but right now makes all values unique instead of just the repeats
count = -1
dummy_list = id_l
for i in id_l:
    count+=1
    for j in dummy_list:
        if i == j:
            id_l[count] = i + (0.0000001 * count)
id_l[:50]

[3652.0,
 2137.0000001,
 1827.0000002,
 1578.0000003,
 1252.0000004,
 1233.0000005,
 1150.0000006,
 1020.0000007,
 898.0000008,
 892.0000009,
 883.000001,
 689.0000011,
 659.0000012,
 581.0000013,
 580.0000014,
 550.0000015,
 544.0000016,
 523.0000017,
 482.0000018,
 469.0000019,
 463.000002,
 424.0000021,
 401.0000022,
 371.0000023,
 371.0000024,
 364.0000025,
 345.0000026,
 345.0000027,
 345.0000028,
 345.0000029,
 344.000003,
 334.0000031,
 334.0000032,
 330.0000033,
 318.0000034,
 299.0000035,
 284.0000036,
 284.0000037,
 283.0000038,
 277.0000039,
 260.000004,
 250.0000041,
 230.0000042,
 228.0000043,
 219.0000044,
 211.0000045,
 210.0000046,
 207.0000047,
 204.0000048,
 201.0000049]

In [11]:
hp_unique_df['id'] = id_l
hp_unique_df.tail(50)

,word,id
11930,fifty-eight).,1.001193
11931,feelings.,1.001193
11932,feelings,1.001193
11933,feeling.,1.001193
11934,"feeling,",1.001193
11935,feather.,1.001194
11936,"feather,",1.001194
11937,feast.,1.001194
11938,"feast,”",1.001194
11939,"feast,",1.001194


In [17]:
word_unique_l = hp_unique_df['word'].to_list()
word_unique_l[:10]

['the', 'and', 'to', 'a', 'Harry', 'of', 'was', 'he', 'in', 'his']

In [21]:
hp_id_dict = {}
for i,j in zip(word_unique_l,id_l):
    hp_id_dict[i] = j
hp_id_dict

{'the': 3652.0,
 'and': 2137.0000001,
 'to': 1827.0000002,
 'a': 1578.0000003,
 'Harry': 1252.0000004,
 'of': 1233.0000005,
 'was': 1150.0000006,
 'he': 1020.0000007,
 'in': 898.0000008,
 'his': 892.0000009,
 '—': 883.000001,
 'had': 689.0000011,
 'said': 659.0000012,
 'you': 581.0000013,
 'at': 580.0000014,
 'it': 550.0000015,
 'on': 544.0000016,
 'that': 523.0000017,
 'as': 482.0000018,
 'He': 469.0000019,
 'I': 463.000002,
 'they': 424.0000021,
 'with': 401.0000022,
 'Stone': 371.0000023,
 'Potter': 371.0000024,
 'but': 364.0000025,
 '|': 345.0000026,
 'Page': 345.0000027,
 'Rowling': 345.0000028,
 'Philosophers': 345.0000029,
 'be': 344.000003,
 'J.K.': 334.0000031,
 '-': 334.0000032,
 'for': 330.0000033,
 'out': 318.0000034,
 'were': 299.0000035,
 'all': 284.0000036,
 'him': 284.0000037,
 'have': 283.0000038,
 'up': 277.0000039,
 'The': 260.000004,
 'Ron': 250.0000041,
 'what': 230.0000042,
 'from': 228.0000043,
 'into': 219.0000044,
 'back': 211.0000045,
 'Hagrid': 210.0000046,
 

In [20]:
words = hp_df['word'].to_list()
words[:10]

['THE',
 'BOY',
 'WHO',
 'LIVED',
 'Mr.',
 'and',
 'Mrs.',
 'Dursley,',
 'of',
 'number']

In [22]:
hp_story_dict = {}
for i in words:
    hp_story_dict[i] = hp_id_dict[i]
hp_story_dict

{'THE': 17.0000569,
 'BOY': 1.0008915,
 'WHO': 1.0007681,
 'LIVED': 1.0009245,
 'Mr.': 79.0000135,
 'and': 2137.0000001,
 'Mrs.': 44.0000248,
 'Dursley,': 6.0001566,
 'of': 1233.0000005,
 'number': 14.0000684,
 'four,': 3.0002593,
 'Privet': 16.0000608,
 'Drive,': 3.0002519,
 'were': 299.0000035,
 'proud': 3.0002546,
 'to': 1827.0000002,
 'say': 52.0000211,
 'that': 523.0000017,
 'they': 424.0000021,
 'perfectly': 5.0001742,
 'normal,': 2.0003341,
 'thank': 4.0002197,
 'you': 581.0000013,
 'very': 160.0000068,
 'much.': 9.0001069,
 'They': 155.0000071,
 'the': 3652.0,
 'last': 61.0000182,
 'people': 74.0000148,
 'you’d': 16.0000607,
 'expect': 11.0000858,
 'be': 344.000003,
 'involved': 3.0003,
 'in': 898.0000008,
 'anything': 46.0000241,
 'strange': 16.0000618,
 'or': 93.0000109,
 'mysterious,': 1.001042,
 'because': 83.0000125,
 'just': 157.0000069,
 'didn’t': 183.0000055,
 'hold': 11.0000902,
 'with': 401.0000022,
 'such': 21.0000468,
 'nonsense.': 1.0010347,
 'Dursley': 43.0000253,

In [30]:
hp_story_df = pd.DataFrame(columns=['word','id'])
hp_story_df.head()

,word,id


In [32]:
hp_story_df['word'] = hp_story_dict.keys()
hp_story_df['id'] = hp_story_dict.values()
hp_story_df.head(50)

,word,id
0,THE,17.000057
1,BOY,1.000892
2,WHO,1.000768
3,LIVED,1.000924
4,Mr.,79.000013
5,and,2137.000000
6,Mrs.,44.000025
7,"Dursley,",6.000157
8,of,1233.000000
9,number,14.000068


In [33]:
hp_story_df['next_id'] = hp_story_df['id'].shift(1)
hp_story_df.head(10)

,word,id,next_id
0,THE,17.000057,NaN
1,BOY,1.000892,17.000057
2,WHO,1.000768,1.000892
3,LIVED,1.000924,1.000768
4,Mr.,79.000013,1.000924
5,and,2137.000000,79.000013
6,Mrs.,44.000025,2137.000000
7,"Dursley,",6.000157,44.000025
8,of,1233.000000,6.000157
9,number,14.000068,1233.000000


In [34]:
model_df = hp_story_df.drop(columns='word')
model_df.head()

,id,next_id
0,17.000057,NaN
1,1.000892,17.000057
2,1.000768,1.000892
3,1.000924,1.000768
4,79.000013,1.000924


In [35]:
model_df.dropna(inplace=True)
model_df.head()

,id,next_id
1,1.000892,17.000057
2,1.000768,1.000892
3,1.000924,1.000768
4,79.000013,1.000924
5,2137.000000,79.000013


In [38]:
# Defines function to split X and y data
def window_data(df, window, feature_col_number, target_col_number):
    # Creates empty X and y list
    X = []
    y = []
    # loops through btc data
    for i in range(len(df) - window - 1):
        # splits the features by grouping window amount of previous close data
        features = df.iloc[i:(i + window), feature_col_number]
        # splits target by taking current close values
        target = df.iloc[(i + window), target_col_number]
        # Adds end of each loops data to respective X and y list
        X.append(features)
        y.append(target)
    # returns X,y as arrays
    return np.array(X), np.array(y).reshape(-1, 1)

In [40]:
# Defines window size
window_size=10

In [41]:
# Runs function to split X and y data
X, y=window_data(model_df, window_size, 0, 1)
# Displays X and y
display(X)
display(y)

array([[1.00089150e+00, 1.00076810e+00, 1.00092450e+00, ...,
        1.23300000e+03, 1.40000684e+01, 3.00025930e+00],
       [1.00076810e+00, 1.00092450e+00, 7.90000135e+01, ...,
        1.40000684e+01, 3.00025930e+00, 1.60000608e+01],
       [1.00092450e+00, 7.90000135e+01, 2.13700000e+03, ...,
        3.00025930e+00, 1.60000608e+01, 3.00025190e+00],
       ...,
       [1.00114650e+00, 1.00076640e+00, 1.00057550e+00, ...,
        1.00112960e+00, 1.00072400e+00, 1.00103120e+00],
       [1.00076640e+00, 1.00057550e+00, 1.00050970e+00, ...,
        1.00072400e+00, 1.00103120e+00, 1.00065070e+00],
       [1.00057550e+00, 1.00050970e+00, 1.00104270e+00, ...,
        1.00103120e+00, 1.00065070e+00, 1.00073940e+00]])

array([[ 3.0002593],
       [16.0000608],
       [ 3.0002519],
       ...,
       [ 1.0010312],
       [ 1.0006507],
       [ 1.0007394]])

In [43]:
split = int(len(X)*0.7)
# Splits train and test data
X_train=X[:split]
X_test=X[split:]
y_train=y[:split]
y_test=y[split:]

In [44]:
X_train=X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test=X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
display(X_train.shape)
display(X_test.shape)

(8377, 10, 1)

(3591, 10, 1)

In [45]:
# Creates neural network
model=Sequential()
# Defines and adds LSTM layer
first_lstm_layer=LSTM(units=30, return_sequences=True)
model.add(first_lstm_layer)
# Randomly drops 20% of defined nodes
first_dropout_layer=Dropout(0.2)
# Defines and adds LSTM layer
model.add(first_dropout_layer)
second_lstm_layer=LSTM(units=30, return_sequences=True)
model.add(second_lstm_layer)
# Randomly drops 20% of defined nodes
second_dropout_layer=Dropout(0.2)
# Defines and adds LSTM layer
model.add(second_dropout_layer)
third_lstm_layer=LSTM(units=30)
model.add(third_lstm_layer)
# Defines and adds output layer with linear activation
output_layer=Dense(units=1, activation='linear')
model.add(output_layer)

2022-08-18 12:19:31.078043: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [47]:
# Compiles model
model.compile(loss='mean_squared_error', optimizer='adam', metrics='mean_squared_error')

In [49]:
# Initially fits model using training data
model.fit(X_train, y_train, epochs=50)

Epoch 1/50
262/262 [==============================] - 12s 17ms/step - loss: 3911.7825 - mean_squared_error: 3911.7825
Epoch 2/50
262/262 [==============================] - 5s 17ms/step - loss: 3857.3799 - mean_squared_error: 3857.3799
Epoch 3/50
262/262 [==============================] - 5s 18ms/step - loss: 3815.3945 - mean_squared_error: 3815.3945
Epoch 4/50
262/262 [==============================] - 5s 17ms/step - loss: 3777.6416 - mean_squared_error: 3777.6416
Epoch 5/50
262/262 [==============================] - 5s 19ms/step - loss: 3743.5178 - mean_squared_error: 3743.5178
Epoch 6/50
262/262 [==============================] - 5s 18ms/step - loss: 3712.5591 - mean_squared_error: 3712.5591
Epoch 7/50
262/262 [==============================] - 5s 18ms/step - loss: 3683.8364 - mean_squared_error: 3683.8364
Epoch 8/50
262/262 [==============================] - 5s 17ms/step - loss: 3655.5098 - mean_squared_error: 3655.5098
Epoch 9/50
262/262 [==============================] - 5s 18ms/s

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 30)            3840      
                                                                 
 dropout (Dropout)           (None, 10, 30)            0         
                                                                 
 lstm_1 (LSTM)               (None, 10, 30)            7320      
                                                                 
 dropout_1 (Dropout)         (None, 10, 30)            0         
                                                                 
 lstm_2 (LSTM)               (None, 30)                7320      
                                                                 
 dense (Dense)               (None, 1)                 31        
                                                                 
Total params: 18,511
Trainable params: 18,511
Non-traina

In [55]:
predictions = model.predict(X_test)
display(predictions)
raveled=predictions.ravel()
display(raveled)
predictions=list(raveled)
predictions[-1]

113/113 [==============================] - 1s 6ms/step


array([[3.4029217],
       [3.4026654],
       [3.4025931],
       ...,
       [3.4025476],
       [3.4025474],
       [3.4025474]], dtype=float32)

array([3.4029217, 3.4026654, 3.4025931, ..., 3.4025476, 3.4025474,
       3.4025474], dtype=float32)

3.4025474

In [56]:
hp_id_dict_2[predictions[-1]]

KeyError: 3.4025474

In [53]:
hp_id_dict_2 = {}
for i,j in zip(word_unique_l,id_l):
    hp_id_dict_2[j] = i
hp_id_dict_2

{3652.0: 'the',
 2137.0000001: 'and',
 1827.0000002: 'to',
 1578.0000003: 'a',
 1252.0000004: 'Harry',
 1233.0000005: 'of',
 1150.0000006: 'was',
 1020.0000007: 'he',
 898.0000008: 'in',
 892.0000009: 'his',
 883.000001: '—',
 689.0000011: 'had',
 659.0000012: 'said',
 581.0000013: 'you',
 580.0000014: 'at',
 550.0000015: 'it',
 544.0000016: 'on',
 523.0000017: 'that',
 482.0000018: 'as',
 469.0000019: 'He',
 463.000002: 'I',
 424.0000021: 'they',
 401.0000022: 'with',
 371.0000023: 'Stone',
 371.0000024: 'Potter',
 364.0000025: 'but',
 345.0000026: '|',
 345.0000027: 'Page',
 345.0000028: 'Rowling',
 345.0000029: 'Philosophers',
 344.000003: 'be',
 334.0000031: 'J.K.',
 334.0000032: '-',
 330.0000033: 'for',
 318.0000034: 'out',
 299.0000035: 'were',
 284.0000036: 'all',
 284.0000037: 'him',
 283.0000038: 'have',
 277.0000039: 'up',
 260.000004: 'The',
 250.0000041: 'Ron',
 230.0000042: 'what',
 228.0000043: 'from',
 219.0000044: 'into',
 211.0000045: 'back',
 210.0000046: 'Hagrid',
 

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(LENGTH_WORD, len(unique_words))))
model.add(Dense(len(unique_words),activation='softmax'))

In [ ]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=2, shuffle=True).history

/Users/robbyodum/opt/anaconda3/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/2
618/618 [==============================] - 338s 541ms/step - loss: 6.8397 - accuracy: 0.1062 - val_loss: 6.6642 - val_accuracy: 0.1222
Epoch 2/2
618/618 [==============================] - 294s 477ms/step - loss: 6.3905 - accuracy: 0.1554 - val_loss: 7.0004 - val_accuracy: 0.1306
